<a href="https://colab.research.google.com/github/herjh0405/DACON_Meal/blob/master/LH_%EA%B3%B5%ED%9C%B4%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 공휴일 인사이트
가설. 공휴일과 휴가자는 큰 연관성이 있을 것이다.

### Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import Library

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
np.random.seed(0)

# 한글 폰트 사용
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'
    
    font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    os.system(font_download_cmd)
    os.system(unzip_cmd)
    
    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    font_name = fm.FontProperties(fname=font_files[0]).get_name()
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

font_download_url = "https://fonts.google.com/download?family=Noto%20Sans%20KR"
change_matplotlib_font(font_download_url)

## Data Preprocessing

### Load Data

In [ ]:
path = '/content/drive/MyDrive/구내식당/water/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
holiday = pd.read_csv(path+'holidays.csv', index_col=0)
corona = pd.read_csv(path+'corona_data.csv')

df = pd.concat([train.iloc[:, :-2], test])
target_df = train.iloc[:, -2:]
df.columns = ['일자', '요일', '정원','휴가자', '출장자', '야근자',\
                 '재택근무자', '조식', '중식', '석식']

### 년, 월, 일 컬럼 생성

In [ ]:
df['재택근무자'] = df['재택근무자'].astype('int')

df['일자'] = pd.to_datetime(df['일자'])
df['년'] = df['일자'].dt.year
df['월'] = df['일자'].dt.month
df['일'] = df['일자'].dt.day
df['월일'] = df['일자'].apply(lambda x : str(x)[5:10])

### 공휴일 데이터 추가
* Only_공휴일 전후 컬럼
* 공휴일+휴일 전후 컬럼 추가

In [ ]:
holiday['date'] = pd.to_datetime(holiday['date'])

df['only_before_holiday'] = df['일자'].apply(lambda x : 1 if (x+dt.timedelta(1) in holiday['date'].tolist()) else 0)
df['only_after_holiday'] = df['일자'].apply(lambda x : 1 if (x-dt.timedelta(1) in holiday['date'].tolist()) else 0)

df['before_holiday'] = df['일자'].apply(lambda x : 1 if (x+dt.timedelta(1) in holiday['date'].tolist())\
                                      or ((x+dt.timedelta(1)).weekday() == 5) or ((x+dt.timedelta(1)).weekday() == 6) else 0)
df['after_holiday'] = df['일자'].apply(lambda x : 1 if (x-dt.timedelta(1) in holiday['date'].tolist())\
                                     or ((x-dt.timedelta(1)).weekday() == 5) or ((x-dt.timedelta(1)).weekday() == 6) else 0)

train = pd.concat([df.iloc[:train.shape[0],:], target_df], axis=1)

### 공휴일 전후로 데이터의 흐름

In [ ]:
plt.figure(figsize=(15, 8))
plt.subplot(131)
plt.boxplot([df[df['only_before_holiday']==1]['휴가자'], df[df['only_after_holiday']==1]['휴가자'],\
             df[(df['only_before_holiday']==1)&(df['only_after_holiday']==1)]['휴가자'], df[(df['only_before_holiday']==0)&(df['only_after_holiday']==0)]['휴가자']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('휴가자 수 평균')
plt.title('휴가와-쉬는날 연관성')

plt.subplot(132)
plt.boxplot([train[train['only_before_holiday']==1]['중식계'], train[train['only_after_holiday']==1]['중식계'],\
             train[(train['only_before_holiday']==1)&(train['only_after_holiday']==1)]['중식계'], train[(train['only_before_holiday']==0)&(train['only_after_holiday']==0)]['중식계']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('중식계 수 평균')
plt.title('중식계-쉬는날 연관성')

plt.subplot(133)
plt.boxplot([train[train['only_before_holiday']==1]['석식계'], train[train['only_after_holiday']==1]['석식계'],\
             train[(train['only_before_holiday']==1)&(train['only_after_holiday']==1)]['석식계'], train[(train['only_before_holiday']==0)&(train['only_after_holiday']==0)]['석식계']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('석식계 수 평균')
plt.title('석식계-쉬는날 연관성')

plt.show()

### 휴일 전후로 데이터의 흐름

In [ ]:
plt.figure(figsize=(15, 8))
plt.subplot(131)
plt.boxplot([df[df['before_holiday']==1]['휴가자'], df[df['after_holiday']==1]['휴가자'],\
             df[(df['before_holiday']==1)&(df['after_holiday']==1)]['휴가자'], df[(df['before_holiday']==0)&(df['after_holiday']==0)]['휴가자']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('휴가자 수 평균')
plt.title('휴가와-쉬는날 연관성')

plt.subplot(132)
plt.boxplot([train[train['before_holiday']==1]['중식계'], train[train['after_holiday']==1]['중식계'],\
             train[(train['before_holiday']==1)&(train['after_holiday']==1)]['중식계'], train[(train['before_holiday']==0)&(train['after_holiday']==0)]['중식계']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('중식계 수 평균')
plt.title('중식계-쉬는날 연관성')

plt.subplot(133)
plt.boxplot([train[train['before_holiday']==1]['석식계'], train[train['after_holiday']==1]['석식계'],\
             train[(train['before_holiday']==1)&(train['after_holiday']==1)]['석식계'], train[(train['before_holiday']==0)&(train['after_holiday']==0)]['석식계']])
plt.xticks(np.arange(1, 5, 1), labels=['쉬는 전날', '쉬는 다음날', '앞뒤', '그냥 평일'])
plt.ylabel('석식계 수 평균')
plt.title('석식계-쉬는날 연관성')

plt.show()